In [1]:
import gzip,sys,struct

In [2]:
train_images = "dataset/train-images-idx3-ubyte.gz"
train_labels = "dataset/train-labels-idx1-ubyte.gz"
t10k_images = "dataset/t10k-images-idx3-ubyte.gz"
t10k_labels = "dataset/t10k-labels-idx1-ubyte.gz"

In [3]:
def read_labels(filename):
    labels=[]
    with gzip.open(filename) as bytestream:
        index=0
        buf=bytestream.read()
        bytestream.close()
        
        #解压数据
        magic,numberOfLabels=struct.unpack_from('>II',buf,index)
        print('magic:',magic,'\n'
              'numberOfLabels:',numberOfLabels)
        
        index=8
        while index<numberOfLabels:
            num=int(struct.unpack_from('B',buf,index)[0])
            labels.append(num)
            index+=1
    return labels

print('train labels >>>>>>>>>>>>>>>>>>')
train_label=read_labels(train_labels)
print(train_label[:100])

print('test labels >>>>>>>>>>>>>>>>>>>>')
t10k_label=read_labels(t10k_labels)
print(t10k_label[:100])

train labels >>>>>>>>>>>>>>>>>>
magic: 2049 
numberOfLabels: 60000
[5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 6, 7, 4, 6, 8, 0, 7, 8, 3, 1]
test labels >>>>>>>>>>>>>>>>>>>>
magic: 2049 
numberOfLabels: 10000
[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9]


In [8]:
def read_img(filename,labels):
    with gzip.open(filename) as bytestream:
        index=0
        buf=bytestream.read()
        bytestream.close()
        
        magic,numberOfImages,rows,col=struct.unpack_from('>IIII',buf,index)
        print('magic:',magic,'\n'
              'numberOfImages:',numberOfImages,'\n'
              'rows:',rows,'\n'
              'col:',col)
        
        index=16
        for i in range(1):
            print('label:',labels[i])
            for x in range(rows):
                for y in range(col):
                    num=int(struct.unpack_from('B',buf,index)[0])
                    if num>100:
                        sys.stdout.write(str(num)+" ")
                    elif num>50:
                        sys.stdout.write(str(num)+"  ")
                    else:
                        sys.stdout.write(str('0   '))
                    index+=1
                sys.stdout.write(str('\n'))
                sys.stdout.write(str('\n'))
            sys.stdout.flush()
    return

read_img(train_images,train_label)

magic: 2051 
numberOfImages: 60000 
rows: 28 
col: 28
label: 5
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   126 136 175 0   166 255 247 127 0   0   0   0   

0   0   0   0   0   0   0   0   0   0   94  154 170 253 253 253 253 253 225 172 253 242 195 64  0   0   0   0   

0   0   0   0   0   0   0   0   238 253 253 253 253 253 253 253 253 251 93  82  82  56  0   0   0   0   0   0   

0   0   0   0   0   0   0

In [16]:
import numpy as np
def read32(bytestream):
    # 由于网络数据的编码是大端，所以需要加上>
    dt = np.dtype(np.int32).newbyteorder('>')
    data = bytestream.read(4)
    return np.frombuffer(data, dt)[0]

def read_label2(filename):
    with gzip.open(filename) as bytestream:
        magic = read32(bytestream)
        numberOfLabels = read32(bytestream)
        labels = np.frombuffer(bytestream.read(numberOfLabels), np.uint8)
        data = np.zeros((numberOfLabels, 10))
        for i in range(100):
            data[i][labels[i]] = 1
        bytestream.close()
    return data

def read_img2(filename):
    # 把文件解压成字节流
    with gzip.open(filename) as bytestream:
        magic = read32(bytestream)
        numberOfImages = read32(bytestream)
        rows = read32(bytestream)
        columns = read32(bytestream)
        images = np.frombuffer(bytestream.read(numberOfImages * rows * columns), np.uint8)
        images.shape = (numberOfImages, rows * columns)
        images = images.astype(np.float32)
        images = np.multiply(images, 1.0 / 255.0)
        bytestream.close()
    return images

trainlabel=read_label2(train_labels)
print(trainlabel)

trainimg=read_img2(train_images)
print(trainimg)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
